In [189]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import spacy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [190]:
# Text cleaning function:

def text_cleaner(text):
    text = re.sub(r'--', ' ', text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [191]:
# Importing and applying cleaning function:

persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

persuasion = re.sub(r'Chapter /d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [192]:
# Prepping the texts to work in spaCy:

nlp = spacy.load('en')

alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Dataframe of the sentences in each text:
sentences = pd.DataFrame(alice_sents + persuasion_sents)

In [193]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(300)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

common_words = set(alicewords + persuasionwords)


In [194]:
common_words = list(common_words)

common_words = common_words[:300]

In [195]:
# Create our data frame with features:

word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


In [196]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9189340813464235

Test set score: 0.8473507148864592


## My Trial - Challenge 0:

In [197]:
alice_doc[10].pos_

'ADJ'

In [198]:
from collections import Counter

def bag_of_ads(text):
    
    ads2 = [token.lower_
           for token in text
           if token.pos_ == 'ADJ'
           if not token.is_stop
           and not token.is_punct]
    
    return [item[0] for item in Counter(ads2).most_common(2000)]

In [199]:
def boa_features(sentences, common_words):
    
        # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to ads, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lower_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lower_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [200]:
# Getting the most common adjectives for each text:
alice_ads = bag_of_ads(alice_doc)
persuasion_ads = bag_of_ads(persuasion_doc)

# Combining theit counts to avoid duplicates:
alice_persuasion_ads = set(alice_ads + persuasion_ads)

# Getting the count per sentence and making a dataframe:
ads_df = boa_features(sentences, alice_persuasion_ads)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


In [201]:
rfc2 = ensemble.RandomForestClassifier()

Y = ads_df['text_source']
X = ads_df.drop(['text_sentence', 'text_source'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size = .5,
                                                    random_state = 0)

In [202]:
rfc2 = rfc2.fit(X_train, y_train)

In [203]:
rfc2.score(X_train, y_train)

0.8239649949511949

In [247]:
rfc2.score(X_test, y_test)

0.7166890982503364

### Now trying to see if adding adverbs would help the accuracy:

In [204]:
from collections import Counter

def bag_of_adverbs(text):
    
    ads2 = [token.lower_
           for token in text
           if token.pos_ == 'ADV'
           if not token.is_stop
           and not token.is_punct]
    
    return [item[0] for item in Counter(ads2).most_common(2000)]

In [205]:
def bo_adverbs_features(sentences, common_words):
    
        # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to ads, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lower_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lower_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [210]:
# Getting the most common adjectives for each text:
alice_adverbs = bag_of_adverbs(alice_doc)
persuasion_adverbs = bag_of_adverbs(persuasion_doc)

# Combining theit counts to avoid duplicates:
alice_persuasion_adverbs = set(alice_adverbs + persuasion_adverbs)

# Getting the count per sentence and making a dataframe:
adverbs_df = bo_adverbs_features(sentences, alice_persuasion_adverbs)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


In [211]:
ads_df.head(3)

,incommoded,injurious,upper,lifeless,extra,nonsensical,eyed,bare,green,alarmed,...,irresolute,open,ambitious,strange,loveliest,good,giddy,thoughtful,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll


In [212]:
adverbs_df.head(3)

,permanently,totally,wildly,upstairs,away,far,equally,softly,familiarly,unproductively,...,close,snappishly,possibly,materially,unfortunately,open,naturally,good,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll


In [261]:
# Combining the adverb and adjective dfs:
ads_df2 = ads_df.drop(['text_source', 'text_sentence'], axis = 1)
combined_df = pd.concat([ads_df2, adverbs_df], axis = 1)

In [262]:
X2 = combined_df.drop(['text_sentence', 'text_source'], axis = 1)
y2 = combined_df['text_source']

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, 
                                                        test_size = .2)

In [263]:
rfc3 = ensemble.RandomForestClassifier()

In [264]:
rfc3.fit(X2_train, y2_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [265]:
rfc3.score(X2_train, y2_train)

0.8435002103491797

In [266]:
rfc.score(X2_test, y2_test)

ValueError: Number of features of the model must match the input. Model n_features is 300 and input n_features is 1537 